# [1] Large VLM-based Vision-Language-Action Models for Robotic Manipulation: A Survey
https://arxiv.org/html/2508.13073v1

# [2] Early Fusion Helps Vision Language Action Models Generalize Better
https://openreview.net/pdf/597ad7d82069689b810bb1d506f1ed3dcfbe2bc1.pdf

Гипотеза: при помощи правильного подобранного смешивания энкодеров можно улучшить понимаю моделью совместного пространства действие-текст-изображение

Тут хочется рассмотреть не только методы, которые используются только для VLA, но и рассмотреть варианты заимствования подходов из исследований мультимодальности для VLM, как например посмотреть, можно ли применить что то похожее на архитектуру OmniFusion, с добавлением адаптеров под конкретные задачи

В [2] показывается то, как можно использовать CLIP в VLA модели, однако он обучается только на основе пар изображений-текст, отсюда хочется попробовать вывести гипотезу, что можно попробовать обучать совместное представление на базе триплета изображение-действие-текст

Как первую инстанцию проверки, как мне кажется, стоит рассмотреть InfoNCE на этих трех модальностей в 2ух видах, а именно: попарный ((изображение-действие, действие-текст и изображение-текст), совместный (изображение-действие-текст)

In [6]:
import torch
import torch.nn.functional as F
from torch import nn

"""
Реализуем код для двух вариантов реализации конгтрастного обучения для получения совместных эмбеддингов
"""


def multi_modal_info_nce_loss(embeddings_list, temperature=0.1, eps=1e-8):
     
    M = len(embeddings_list) 
    B, D = embeddings_list[0].shape
    for e in embeddings_list:
        assert e.shape == (B, D)
 
    embeddings = [F.normalize(e, dim=1) for e in embeddings_list]   
 
    z = torch.cat(embeddings, dim=0)
    device = z.device
    N = z.shape[0] 
    sim = torch.matmul(z, z.T) / temperature
 
    diag_mask = torch.eye(N, device=device).bool()
 
    idx = torch.arange(N, device=device)
    sample_idx = idx % B 
    same_sample = sample_idx.unsqueeze(0) == sample_idx.unsqueeze(1)   
    positives_mask = same_sample & (~diag_mask)  
    exp_sim = torch.exp(sim) * (~diag_mask).float() 
    numerator = (exp_sim * positives_mask.float()).sum(dim=1)  
 
    denominator = exp_sim.sum(dim=1) + eps  
 
    loss_i = -torch.log((numerator + eps) / denominator) 
    return loss_i.mean()

 
def nt_xent_pairwise(z1, z2, temperature=0.1): 
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    logits = torch.matmul(z1, z2.T) / temperature 
    labels = torch.arange(z1.size(0), device=z1.device) 
    loss1 = F.cross_entropy(torch.cat([logits,], dim=1), labels) 
    return loss1


    
class DummyEncoder(nn.Module):
    def __init__(self, input_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )
    def forward(self, x):
        return self.net(x)
 
image_encoder = DummyEncoder(input_dim=2048, out_dim=256)
text_encoder  = DummyEncoder(input_dim=768,  out_dim=256)
state_encoder = DummyEncoder(input_dim=1024, out_dim=256)
 
B = 32
image_feats = torch.randn(B, 2048)
text_feats  = torch.randn(B, 768)
state_feats = torch.randn(B, 1024)
 
z_img = image_encoder(image_feats)
z_txt = text_encoder(text_feats)
z_act = state_encoder(state_feats)

loss = multi_modal_info_nce_loss([z_img, z_txt, z_act], temperature=0.07)
loss_ta = nt_xent_pairwise(z_txt, z_act, temperature=0.07)
loss_it = nt_xent_pairwise(z_img, z_txt, temperature=0.07)
loss_ai = nt_xent_pairwise(z_img, z_act, temperature=0.07)
print("Loss :", loss.item())
print("Pairwise avg. loss:", (loss_ta.item() + loss_it.item() + loss_ai.item()) / 3)

opt = torch.optim.Adam(list(image_encoder.parameters()) +
                       list(text_encoder.parameters()) +
                       list(state_encoder.parameters()), lr=1e-4)

opt.zero_grad()
loss.backward()
opt.step()


Loss : 7.7399749755859375
Pairwise avg. loss: 3.822693427403768


На этом этапе мы можем только посмотреть на то, что в общем Loss на совмещение 3 модальностей выше, чем если брать средний попарный

Также есть вариант с модальными адаптерами, например как это реализовано в OmniFusion у AIRI, приведем два примера адаптеров

In [7]:
import math

import torch
import torch.nn as nn


class MLPAdapter(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.mlp = nn.Sequential(
            nn.Linear(self.in_dim, self.out_dim), nn.GELU(), nn.Linear(out_dim, out_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.mlp(x)
        return out
 


class QFormer(nn.Module):
    def __init__(self,
                 visual_hidden_dim: int,
                 query_dim: int,
                 num_queries: int,
                 transformer_hidden_dim: int,
                 num_transformer_layers: int,
                 num_heads: int):
        super(QFormer, self).__init__()

        # Learnable query vectors
        self.queries = nn.Parameter(torch.randn(num_queries, query_dim))

        # Linear projection from visual encoder dimension to transformer hidden dimension
        self.visual_projection = nn.Linear(visual_hidden_dim, transformer_hidden_dim)

        # Transformer encoder to process queries and visual features
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=transformer_hidden_dim,
                nhead=num_heads,
                dim_feedforward=4 * transformer_hidden_dim,
                dropout=0.1,
                activation='relu'
            ),
            num_layers=num_transformer_layers
        )

        # Output projection to get final query embeddings
        self.output_projection = nn.Linear(transformer_hidden_dim, query_dim)

    def forward(self, visual_features: torch.Tensor):
        """
        visual_features: Tensor of shape (batch_size, num_patches, visual_hidden_dim)
        """
        batch_size = visual_features.size(0)

        # Project visual features to the transformer hidden dimension
        projected_visual_features = self.visual_projection(
            visual_features)  # Shape: (batch_size, num_patches, transformer_hidden_dim)

        # Repeat the queries across the batch
        queries = self.queries.unsqueeze(0).expand(batch_size, -1, -1)  # Shape: (batch_size, num_queries, query_dim)

        # Concatenate queries and visual features
        combined_features = torch.cat([queries, projected_visual_features],
                                      dim=1)  # Shape: (batch_size, num_queries + num_patches, transformer_hidden_dim)

        # Apply the transformer
        transformed_features = self.transformer(combined_features.permute(1, 0,
                                                                          2))  # Shape: (num_queries + num_patches, batch_size, transformer_hidden_dim)
        transformed_features = transformed_features.permute(1, 0,
                                                            2)  # Shape: (batch_size, num_queries + num_patches, transformer_hidden_dim)

        # Extract the transformed queries
        transformed_queries = transformed_features[:, :self.queries.size(0),
                              :]  # Shape: (batch_size, num_queries, transformer_hidden_dim)

        # Project the transformed queries to the query dimension
        output_queries = self.output_projection(transformed_queries)  # Shape: (batch_size, num_queries, query_dim)

        return output_queries


Последним пунктом из тех экспериментов, которые интересно произвести назовем смешивание эмбеддингов при помощи взвешенного сложения, кросс attn и подобных алгоритмов, по итогу работы которых мы получаем общий вектор представления для двух модальностей

In [9]:
import torch
import torch.nn as nn

class WeightedSumMixer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()

        self.weight_1 = nn.Parameter(torch.randn(1,))
        self.weight_2 = nn.Parameter(torch.randn(1,))

    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        z = self.weight_1 * x1 + self.weight_2 + x2
        return z

Итогом проведения этих экспериментов в идеале стала бы статистика того, насколько нам может помочь смешивание эмбеддингов в VLA, какие идеи можно позаимствовать и как сделать устойчивое пространства общего представления текст-действие-изображение